In [1]:
import pandas as pd
import os
from datetime import datetime
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller
import plotly.express as px
register_matplotlib_converters()

### Read CSV files

In [2]:
cleaned_weather_df = pd.read_csv('data/region/vietnam/cleaned_weather.csv', index_col=0, parse_dates=True)
cleaned_air_df = pd.read_csv('data/region/vietnam/cleaned_air.csv', index_col=0, parse_dates=True)

interpolated_weather_df = pd.read_csv('data/region/vietnam/interpolated_weather.csv', index_col=0, parse_dates=True)
interpolated_air_df = pd.read_csv('data/region/vietnam/interpolated_air.csv', index_col=0, parse_dates=True)

dropped_weather_df = pd.read_csv('data/region/vietnam/dropped_weather.csv', index_col=0, parse_dates=True)
dropped_air_df = pd.read_csv('data/region/vietnam/dropped_air.csv', index_col=0, parse_dates=True)

Data overview

In [7]:
print(cleaned_weather_df.info(verbose=False))
print(cleaned_air_df.info(verbose=False))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2155488 entries, 2020-11-28 00:00:00 to 2024-11-15 07:00:00
Columns: 9 entries, province to wind_direction_10m
dtypes: float64(8), object(1)
memory usage: 164.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2155488 entries, 2020-11-28 00:00:00 to 2024-11-15 07:00:00
Columns: 8 entries, province to aqi
dtypes: float64(7), object(1)
memory usage: 148.0+ MB
None


In [8]:
print(interpolated_weather_df.info(verbose=False))
print(interpolated_air_df.info(verbose=False))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2189880 entries, 2020-11-28 00:00:00 to 2024-11-15 07:00:00
Columns: 9 entries, province to wind_direction_10m
dtypes: float64(8), object(1)
memory usage: 167.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2189880 entries, 2020-11-28 00:00:00 to 2024-11-15 07:00:00
Columns: 8 entries, province to aqi
dtypes: float64(7), object(1)
memory usage: 150.4+ MB
None


In [9]:
print(dropped_weather_df.info(verbose=False))
print(dropped_air_df.info(verbose=False))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2114469 entries, 2020-11-28 00:00:00 to 2024-11-14 07:00:00
Columns: 9 entries, province to wind_direction_10m
dtypes: float64(8), object(1)
memory usage: 161.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2114469 entries, 2020-11-28 00:00:00 to 2024-11-14 07:00:00
Columns: 8 entries, province to aqi
dtypes: float64(7), object(1)
memory usage: 145.2+ MB
None


In [12]:
print(cleaned_weather_df.describe().transpose())
print(cleaned_air_df.describe().transpose())


                          count        mean        std    min     25%     50%  \
temperature_2m        2155488.0   25.138100   4.820720    3.4    22.7    25.6   
relative_humidity_2m  2155488.0   80.546117  14.389070   12.0    72.0    84.0   
dew_point_2m          2155488.0   21.244726   4.626084    0.0    19.3    22.9   
precipitation         2155488.0    0.253754   1.020248    0.0     0.0     0.0   
surface_pressure      2155488.0  997.630910  29.248069  844.1  1001.2  1007.4   
cloud_cover           2155488.0   74.365158  34.922208    0.0    49.0    97.0   
wind_speed_10m        2155488.0    8.873095   5.469387    0.0     4.7     7.6   
wind_direction_10m    2155488.0  164.464754  97.756696    0.0    86.0   151.0   

                         75%     max  
temperature_2m          28.2    43.0  
relative_humidity_2m    92.0   100.0  
dew_point_2m            24.4    32.9  
precipitation            0.1    77.9  
surface_pressure      1011.0  1033.5  
cloud_cover            100.0   100.0

In [13]:
print(interpolated_weather_df.describe().transpose())
print(interpolated_air_df.describe().transpose())

                          count        mean        std    min     25%     50%  \
temperature_2m        2189880.0   25.088331   4.852710    3.4    22.7    25.6   
relative_humidity_2m  2189880.0   80.565587  14.385831   12.0    72.0    84.0   
dew_point_2m          2189880.0   21.201293   4.701071    0.0    19.3    22.8   
precipitation         2189880.0    0.253084   1.017751    0.0     0.0     0.0   
surface_pressure      2189880.0  997.686957  29.250029  844.1  1001.2  1007.5   
cloud_cover           2189880.0   74.404775  34.831824    0.0    49.0    97.0   
wind_speed_10m        2189880.0    8.866432   5.461597    0.0     4.7     7.6   
wind_direction_10m    2189880.0  164.406894  97.826538    0.0    86.0   151.0   

                         75%     max  
temperature_2m          28.2    43.0  
relative_humidity_2m    92.0   100.0  
dew_point_2m            24.4    32.9  
precipitation            0.1    77.9  
surface_pressure      1011.0  1033.5  
cloud_cover            100.0   100.0

In [14]:
print(dropped_weather_df.describe().transpose())
print(dropped_air_df.describe().transpose())

                          count        mean        std    min     25%     50%  \
temperature_2m        2114469.0   25.242884   4.726125    3.6    22.9    25.6   
relative_humidity_2m  2114469.0   80.671114  14.303192   12.0    72.0    84.0   
dew_point_2m          2114469.0   21.373574   4.450432   -0.0    19.5    22.9   
precipitation         2114469.0    0.255999   1.026076    0.0     0.0     0.0   
surface_pressure      2114469.0  997.523627  29.198509  844.1  1001.1  1007.4   
cloud_cover           2114469.0   74.381415  34.891074    0.0    49.0    97.0   
wind_speed_10m        2114469.0    8.833199   5.426990    0.0     4.7     7.6   
wind_direction_10m    2114469.0  164.912068  97.185822    0.0    88.0   152.0   

                         75%     max  
temperature_2m          28.3    43.0  
relative_humidity_2m    92.0   100.0  
dew_point_2m            24.5    32.9  
precipitation            0.1    77.9  
surface_pressure      1010.9  1031.6  
cloud_cover            100.0   100.0

### Save line plots for each attribute in weather data and air data

In [3]:
def savePlotAttributes(df, dir, resample_mode=None):
    for attribute in df.iloc[:,1:].columns:
        fig = plt.figure(figsize=(16, 63 * 3))
        fig.suptitle(attribute, fontsize=20, y=0.95)
        
        provinces = df['province'].drop_duplicates().values
        n_provinces = len(provinces)
        
        for i, province in enumerate(provinces):
            province_df = df[df['province'] == province]
            if resample_mode:
                data = province_df[attribute].resample(resample_mode).mean()
            else:
                data = province_df[attribute]
            
            ax = plt.subplot2grid((n_provinces, 1), (i, 0), rowspan=1, colspan=1)
            ax.plot(data)
            ax.set_title(f'{i + 1}. {province}', fontsize=16)
    
            for year in range(2021, 2026):
                ax.axvline(datetime(year, 1, 1), linestyle='--', color='k', alpha=0.5)
        
        plt.tight_layout(rect=[0, 0, 1, 0.95], pad=2)
        plt.subplots_adjust(hspace=0.8)
        file_name = os.path.join(dir, f"{attribute}.png")
        plt.savefig(file_name, format="png", dpi=300, bbox_inches="tight")
        plt.close()


In [ ]:
savePlotAttributes(cleaned_weather_df, 'plots/weather/cleaned')
savePlotAttributes(cleaned_air_df, 'plots/air/cleaned')

In [5]:
savePlotAttributes(interpolated_weather_df, 'plots/weather/interpolated')
savePlotAttributes(interpolated_air_df, 'plots/air/interpolated')

In [6]:
savePlotAttributes(dropped_weather_df, 'plots/weather/dropped')
savePlotAttributes(dropped_air_df, 'plots/air/dropped')

### Save ACF and PACF plots for each attribute in weather data and air data

In [3]:
def savePlotACFandPACF(df, dir, acf_lags = 25, pacf_lags = 25, resample_mode = None):
    for attribute in df.iloc[:,1:].columns:
        provinces = df['province'].drop_duplicates().values
        n_provinces = len(provinces)

        fig, axes = plt.subplots(n_provinces, 2, figsize=(15, 3 * n_provinces), dpi=80)
        fig.suptitle(f'ACF and PACF for {attribute}', fontsize=20, y=0.95)

        for i, province in enumerate(provinces):
            province_df = df[df['province'] == province]
            if resample_mode:
                data = province_df[attribute].resample(resample_mode).mean()
            else:
                data = province_df[attribute]

            ax_acf = axes[i, 0]
            ax_pacf = axes[i, 1]

            plot_acf(data, ax=ax_acf, lags=acf_lags, title=f'{i+1}. {province} ACF')
            plot_pacf(data, ax=ax_pacf, lags=pacf_lags, title=f'{i+1}. {province} PACF')
            
        plt.tight_layout(rect=[0, 0, 1, 0.95], pad=2)
        file_name = os.path.join(dir, f"{attribute}_acf_and_pacf.png")
        plt.savefig(file_name, format="png", dpi=300, bbox_inches="tight")
        plt.close()

In [4]:
savePlotACFandPACF(cleaned_weather_df, 'plots/weather/cleaned', 25, 25)
savePlotACFandPACF(cleaned_air_df, 'plots/air/cleaned', 25, 25)

In [ ]:
savePlotACFandPACF(interpolated_weather_df, 'plots/weather/interpolated', 25, 25)
savePlotACFandPACF(interpolated_air_df, 'plots/air/interpolated', 25, 25)

In [ ]:
savePlotACFandPACF(dropped_weather_df, 'plots/weather/dropped', 50, 50)
savePlotACFandPACF(dropped_air_df, 'plots/air/dropped', 50, 50)

### Augmented Dickey-Fuller unit root test for stationarity

In [3]:
def perform_adf_test(df, resample_mode = None):
    non_stationary_dict = {}
    for attribute in df.iloc[:,1:].columns:
        provinces = df['province'].drop_duplicates().values
        for i, province in enumerate(provinces):
            province_df = df[df['province'] == province]
            if resample_mode:
                data = province_df[attribute].resample(resample_mode).mean()
            else:
                data = province_df[attribute]
            result = adfuller(data)
            if result[1] >= 0.05:
                if attribute not in non_stationary_dict:
                    non_stationary_dict[attribute] = []
                non_stationary_dict[attribute].append(province)
    if non_stationary_dict:
        print('Non-stationary attribute(s) found:')
        print(non_stationary_dict)
    else:
        print('All attributes are stationary')

In [ ]:
perform_adf_test(cleaned_weather_df)

All attributes are stationary


In [ ]:
perform_adf_test(cleaned_air_df)

All attributes are stationary


Since ADF test for time series with gaps might be inaccurate, will also test on interpolated data

In [ ]:
perform_adf_test(interpolated_weather_df)

All attributes are stationary


In [ ]:
perform_adf_test(interpolated_air_df)

All attributes are stationary


ADF test on dropped data to see if more gaps will cause non-stationrity

In [4]:
perform_adf_test(dropped_weather_df)

All attributes are stationary


In [5]:
perform_adf_test(dropped_air_df)

All attributes are stationary


Conclusion: All three types of data (cleaned, dropped, interpolated) are stationary

### Correlation matrix between air attributes and weather attributes

Set multi-index for correlation

In [ ]:
def plotCorrelationMatrix(df1, df2, title):
    df1 = df1.reset_index().set_index(['time', 'province'])
    df2 = df2.reset_index().set_index(['time', 'province'])
    correlation_matrix = pd.DataFrame(index=df1.columns, columns=df2.columns)
    for attr1 in df1.columns:
        for attr2 in df2.columns:
            correlation_matrix.loc[attr1, attr2] = df1[attr1].corr(df2[attr2])
    fig = px.imshow(
    correlation_matrix,
    labels=dict(color="Correlation"),
    color_continuous_scale='PRGn',
    title=title
    )
    fig.show()

In [ ]:
plotCorrelationMatrix(interpolated_weather_df, interpolated_air_df, "Interpolated data correlation")

Correlation of data with no interpolation

In [ ]:
plotCorrelationMatrix(cleaned_weather_df, cleaned_air_df, "Cleaned data correlation")

Correlation of dropped data

In [ ]:
plotCorrelationMatrix(dropped_weather_df, dropped_air_df, "Cleaned data correlation")

Conclusion: Correlation for three types of data (cleaned, dropped, interpolated) are roughly the same

### STL Decomposition + Residuals plot for outlier detection

In [ ]:
def saveResidualsAttributes(df, dir, resample_mode=None):
    for attribute in df.iloc[:,1:].columns:
        fig = plt.figure(figsize=(12, 63 * 4))
        fig.suptitle(attribute, fontsize=20, y=0.95)
        
        provinces = df['province'].drop_duplicates().values
        n_provinces = len(provinces)
        
        for i, province in enumerate(provinces):
            province_df = df[df['province'] == province]
            if resample_mode:
                data = province_df[attribute].resample(resample_mode).mean()
            else:
                data = province_df[attribute]
            
            ax = plt.subplot2grid((n_provinces, 1), (i, 0), rowspan=1, colspan=1)
            stl = STL(data, period=24)
            res = stl.fit()
            ax.plot_date(res.resid.index, res.resid.values, color='black',linestyle='--')
            ax.set_title(f'{i + 1}. {province}', fontsize=16)
            for year in range(2021, 2026):
                ax.axvline(datetime(year, 1, 1), linestyle='--', color='k', alpha=0.5)
        
        plt.tight_layout(rect=[0, 0, 1, 0.95], pad=2)
        plt.subplots_adjust(hspace=0.8)
        file_name = os.path.join(dir, f"{attribute}_residuals.png")
        plt.savefig(file_name, format="png", dpi=300, bbox_inches="tight")
        plt.close()

In [ ]:
saveResidualsAttributes(cleaned_air_df,'plots/air/cleaned')
saveResidualsAttributes(cleaned_weather_df,'plots/weather/cleaned')

In [ ]:
saveResidualsAttributes(interpolated_air_df,'plots/air/interpolated')
saveResidualsAttributes(interpolated_weather_df,'plots/weather/interpolated')

In [ ]:
saveResidualsAttributes(dropped_air_df,'plots/air/dropped')
saveResidualsAttributes(dropped_weather_df,'plots/weather/dropped')